Referências:
    
- https://redib.org/Record/oai_articulo2448895-utilizando-o-m%C3%A9todo-todim-para-avaliar-melhores-empresas-para-trabalhar
- https://www.marinha.mil.br/spolm/sites/www.marinha.mil.br.spolm/files/126482.pdf
- http://www.abepro.org.br/biblioteca/ENEGEP1998_ART332.pdf
- https://www.mdpi.com/2227-9717/10/3/609
- https://coin-or.github.io/pulp/main/includeme.html
- http://www.optimization-online.org/DB_FILE/2011/09/3178.pdf
- https://machinelearninggeek.com/solving-linear-programming-using-python-pulp/
- https://www.analyticsvidhya.com/blog/2017/10/linear-optimization-in-python/

In [206]:
import re
import random

import pulp

import pandas as pd
import numpy as np

from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [207]:
fiis = pd.read_csv('./bases/fiis_data.csv')

In [208]:
fiis.head(5)

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média,Endereço
0,ABCP11,Shoppings,"73,29","6,97%","7,57%","0,80",894.419.000,90.469,1,"12.537,30","1.019,81","8,13%","7,50%","Avenida Industrial, 600, Centro, Santo André - SP"
1,AEFI11,Outros,"174,90","6,97%","0,00%","1,24",353.582.000,0,6,"4.065,62","300,71","7,40%","0,00%","Avenida São Luís Rei de França, 32, Turu - São..."
2,AFCR11,Híbrido,"103,15","13,16%","7,96%","1,01",156.193.000,0,0,"0,00","0,00","0,00%","0,00%",NaN
3,AFHI11,Títulos e Val. Mob.,"100,88","6,14%","11,07%","1,05",180.340.000,863.145,0,"0,00","0,00","0,00%","0,00%",NaN
4,AFOF11,Títulos e Val. Mob.,"92,20","5,09%","10,78%","0,91",63.727.900,95.373,0,"0,00","0,00","0,00%","0,00%",NaN


In [209]:
filters = [
#     {'on' : 'feature', 'min' : 0.0, 'max' : '0.0'}
    {'on' : 'Liquidez', 'min' : 10000},
    {'on' : 'P/VP', 'min' : 0.8}
]

In [276]:
fiis.shape[0]

160

In [211]:
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

def excel_style(col, row=0):
    """ Convert given row and column number to an Excel-style cell name. """
    result = []
    while col:
        col, rem = divmod(col-1, 26)
        result[:0] = LETTERS[rem]
    return ''.join(result)# + str(row)

In [212]:
def convert_to_float(str_val, default=0.0):
    try:
        str_val = re.sub('[^0-9\.,]','', str_val)
        str_val = str_val.replace('.', '').replace(',', '.')
        return float(str_val)
    except Exception as e:
        print(e, str_val)
        return default

In [213]:
def apply_filter(row, filters):
    flag = True
    for f in filters:
        if('min' in f):
            flag = flag and row[f['on']] >= f['min']
    return flag

---

In [214]:
columns = [
    'Cotação',
    'FFO Yield',
    'Dividend Yield',
    'P/VP',
    'Valor de Mercado',
    'Liquidez',
    'Preço do m2',
    'Aluguel por m2',
    'Cap Rate',
    'Vacância Média'
]
for col in columns:
    fiis[col] = fiis[col].apply(lambda v : convert_to_float(v))
#     print('all good with: ', col)

In [215]:
f_ = fiis.apply(lambda row: apply_filter(row, filters), axis=1)
fiis = fiis[f_].reset_index(drop=True)

In [246]:
fiis['Segmento'].unique()

array(['Shoppings', 'Títulos e Val. Mob.', 'Híbrido',
       'Lajes Corporativas', 'Outros', 'Logística', nan, 'Hospital',
       'Residencial', 'Hotel'], dtype=object)

In [216]:
fiis.head(5)

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média,Endereço
0,ABCP11,Shoppings,73.29,6.97,7.57,0.80,894419000.0,90469.0,1,12537.30,1019.81,8.13,7.5,"Avenida Industrial, 600, Centro, Santo André - SP"
1,AFHI11,Títulos e Val. Mob.,100.88,6.14,11.07,1.05,180340000.0,863145.0,0,0.00,0.00,0.00,0.0,NaN
2,AFOF11,Títulos e Val. Mob.,92.20,5.09,10.78,0.91,63727900.0,95373.0,0,0.00,0.00,0.00,0.0,NaN
3,ALZR11,Híbrido,113.88,5.83,6.51,1.05,674399000.0,1149200.0,13,5283.60,356.09,6.74,0.0,"Rua Fritz Reiman, 628, Itapegica, Guarulhos, S..."
4,ARCT11,Híbrido,106.30,4.94,14.41,1.06,305992000.0,2247110.0,9,404.07,8.68,2.15,0.0,"Rodovia GO-070, Km 2 Qd. CH, Lt. 444, , Goiâni..."


In [217]:
variables = [excel_style(i) for i in list(range(fiis.shape[0]+1))[1:]]

In [218]:
pulp_variables = [eval(f"LpVariable(name='{i}', lowBound=0, cat='Integer')") for i in variables]

In [264]:
cotacao = list(fiis['Cotação'])
expr_cotacao = sum([v*p for v, p in zip(pulp_variables, cotacao)])

profit = list(fiis['Dividend Yield'])
expr_profit = sum([v*c for v, c in zip(pulp_variables, profit)])

expr_diversification = sum([1 if v >= 0.1 else 0 for v in pulp_variables])

In [220]:
# type(cotacao[0])
# convert_to_float(cotacao[0])
# cotacao[0]

---

In [265]:
BUDGET = 100000.00

In [266]:
# 1. Create the model
model = LpProblem(name="portifolio", sense=LpMaximize)

In [267]:
# 2. Add the constraints to the model
model += (expr_cotacao <= BUDGET, "budget_ub_constraint")

In [268]:
# 3. Add the objective function to the model
model += expr_profit + expr_diversification

In [269]:
# 4. Solve the problem
status = model.solve()

In [270]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
print('-'*50)
# for var in model.variables():
#     idx = variables.index(var.name)
#     alimento = smdf.loc[idx, 'food']
#     print(f"{var.name}:\t{var.value()}\t- {alimento}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 208852.0
--------------------------------------------------
budget_ub_constraint: -1.75


In [275]:
model

portifolio:
MAXIMIZE
7.57*A + 7.23*AA + 10.78*AB + 11.52*AC + 12.83*AD + 11.97*AE + 10.05*AF + 15.38*AG + 8.99*AH + 8.87*AI + 6.9*AJ + 10.97*AK + 8.97*AL + 9.11*AM + 14.55*AN + 11.42*AO + 13.75*AP + 2.06*AQ + 12.11*AR + 8.85*AS + 3.51*AT + 10.33*AU + 16.86*AV + 8.56*AW + 7.49*AX + 12.78*AY + 16.35*AZ + 11.07*B + 10.19*BA + 5.88*BB + 10.69*BC + 8.95*BD + 10.02*BE + 8.62*BF + 6.04*BG + 7.8*BH + 8.93*BI + 12.68*BJ + 7.98*BK + 7.1*BL + 6.91*BM + 10.65*BN + 9.84*BO + 12.45*BP + 7.61*BQ + 3.47*BR + 2.29*BS + 5.77*BT + 13.87*BU + 5.06*BV + 4.15*BW + 9.54*BX + 26.4*BY + 9.93*BZ + 10.78*C + 7.04*CA + 14.33*CB + 14.52*CC + 6.68*CD + 15.61*CE + 8.99*CF + 7.86*CG + 6.94*CH + 6.02*CI + 11.12*CJ + 8.43*CK + 7.76*CL + 12.11*CM + 12.93*CN + 12.15*CO + 10.35*CP + 10.26*CQ + 4.88*CR + 11.02*CS + 10.38*CT + 9.12*CU + 3.34*CV + 11.81*CW + 5.91*CX + 11.8*CY + 12.04*CZ + 6.51*D + 13.01*DA + 9.36*DB + 12.57*DC + 44.86*DD + 12.91*DE + 6.62*DF + 9.32*DG + 13.85*DH + 10.23*DI + 10.83*DJ + 8.56*DK + 14.97*DL + 9

In [271]:
designed_portifolio = []
for i, var in enumerate(model.variables()):
    idx = variables.index(var.name)
    value = var.value()
    if(value > 0):
        print(f"{idx} - {var.name}: {value} - {fiis.loc[idx, 'Papel']} - {fiis.loc[idx, 'Segmento']}")
        designed_portifolio.append({
            'asset': fiis.loc[idx, 'Papel'],
            'qnt' : value,
            'idx' : idx,
            'segment' : fiis.loc[idx, 'Segmento'],
            'value' : value * fiis.loc[idx, 'Cotação'],
            'cotation' : fiis.loc[idx, 'Cotação'],
            'dy' : fiis.loc[idx, 'Dividend Yield'],
            'p/vp' : fiis.loc[idx, 'P/VP'],
        })

76 - BY: 7905.0 - KINP11 - Residencial


In [272]:
designed_portifolio

[{'asset': 'KINP11',
  'qnt': 7905.0,
  'idx': 76,
  'segment': 'Residencial',
  'value': 99998.25,
  'cotation': 12.65,
  'dy': 26.4,
  'p/vp': 1.79}]

In [274]:
model.objective.value() - (26.4 * 7905)

160.0